In [1]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
sys.path.append("codes/modules")

computer =  bfn1003
working directory set to /home/hbkoziel/awi-models


In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
import colormaps as cmaps
from matplotlib.colors import ListedColormap

In [3]:
# PLOT CONFIGURATION #

# CSV export
export_csv = True
# choose simulation
runid='Arc12'
# choose date
first_year = 1990
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = True

In [4]:
# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [5]:
mesh = fesom_mesh(meshpath, get3d = get3d)

Load 2d part of the grid in 3 second(s)
Load 3d part of the grid in 36 seconds


In [6]:
def dens0(T,S):
    T68 = T * 1.00024
    a0,a1,a2,a3,a4,a5 = 999.842594, 6.793952e-2, -9.095290e-3, 1.001685e-4, -1.120083e-6, 6.536332e-9
    sw_smow = a0 + (a1 + (a2 + (a3 + (a4 + a5*T68)*T68)*T68)*T68)*T68
    #     UNESCO 1983 eqn(13) p17.
    
    b0, b1, b2, b3, b4 =  8.24493e-1, -4.0899e-3, 7.6438e-5, -8.2467e-7, 5.3875e-9
    c0, c1, c2 = -5.72466e-3, 1.0227e-4, -1.6546e-6
    d0 = 4.8314e-4
    dens = sw_smow + (b0 + (b1 + (b2 + (b3 + b4*T68)*T68)*T68)*T68)*S + (c0 + (c1 + c2*T68)*T68)*S*np.sqrt(S) + d0*S**2
    return dens

In [7]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][4,:]
         temp    = f.variables['temp'][4,:]
     else:
         salt    = f.variables['salt'][45:59,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][45:59,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_APR_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_APR_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w

In [8]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][5,:]
         temp    = f.variables['temp'][5,:]
     else:
         salt    = f.variables['salt'][60:74,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][60:74,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_MAY_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_MAY_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w

In [9]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][6,:]
         temp    = f.variables['temp'][6,:]
     else:
         salt    = f.variables['salt'][75:89,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][75:89,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_JUN_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_JUN_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w

In [10]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][7,:]
         temp    = f.variables['temp'][7,:]
     else:
         salt    = f.variables['salt'][90:105,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][90:105,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_JUL_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_JUL_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w

In [11]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][8,:]
         temp    = f.variables['temp'][8,:]
     else:
         salt    = f.variables['salt'][106:120,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][106:120,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_AUG_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_AUG_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w

In [12]:
# initlialyze
STRAT = np.zeros((len(years),len(mesh.x2)))

# loop over years
for ind in range(0,len(years)):
     print years[ind]
     ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
     f      = Dataset(ncfile, 'r')
     if years[ind]<2000:
         salt    = f.variables['salt'][9,:]
         temp    = f.variables['temp'][9,:]
     else:
         salt    = f.variables['salt'][121:135,:].mean(axis=0)#75:135
         temp    = f.variables['temp'][121:135,:].mean(axis=0)
            
     dens = dens0(temp,salt)
     del salt
     del temp
     dens_0m        = fesom2depth(0, dens, mesh)
     dens_100m      = fesom2depth(100, dens, mesh)
     STRAT[ind,:]= (dens_100m-dens_0m)
     del dens_0m
     del dens_100m   
     
data2 = STRAT.mean(axis=0)
print 'Number of nans in tracer: ',np.count_nonzero(np.isnan(data2))
print 'Number of inf in tracer: ',np.count_nonzero(np.isinf(data2))
print 'Mean of surface: ',np.mean(data2[0:len(mesh.x2)])
print 'Max and min: ',np.max(data2),np.min(data2)

if export_csv == True:
    np.savetxt(outputpath+'STRAT_SEP_'+str(first_year)+'_'+str(last_year)+'_mean.csv', data2, delimiter=";")
    np.savetxt(outputpath+'STRAT_SEP_'+str(first_year)+'_'+str(last_year)+'_trend.csv', STRAT, delimiter=";")
    print 'exporting done'

1990
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1991
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1992
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1993
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1994
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1995
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1996
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1997
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1998
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
1999
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 will be used
2000
For depth 0 model level -0.0 will be used
For depth 100 model level 100.0 w